In [22]:
from spuriosity_metrics import *
from compute_feature_activations import *
from collections import OrderedDict

In [15]:
model_path = '../models/robust_resnet50.pth'
device = 'cpu'
architecture = 'resnet50'

In [13]:
l = torch.load(model_path, map_location=torch.device(device))['model']

In [14]:
l

OrderedDict([('module.normalizer.new_mean',
              tensor([[[0.4850]],
              
                      [[0.4560]],
              
                      [[0.4060]]])),
             ('module.normalizer.new_std',
              tensor([[[0.2290]],
              
                      [[0.2240]],
              
                      [[0.2250]]])),
             ('module.model.conv1.weight',
              tensor([[[[ 2.8998e-03, -8.2329e-04, -2.5784e-03,  ..., -3.1941e-03,
                         -2.4553e-04,  3.4729e-03],
                        [-2.8723e-03, -7.2638e-03, -8.0492e-03,  ..., -8.0426e-03,
                         -5.0311e-03,  2.6362e-03],
                        [ 5.7547e-03,  2.7930e-03, -1.7734e-04,  ...,  3.9307e-03,
                          1.0331e-02,  1.5862e-02],
                        ...,
                        [ 5.9958e-02,  1.0286e-01,  1.3715e-01,  ...,  1.5074e-01,
                          1.0591e-01,  5.6787e-02],
                        [-3.476

In [16]:
model = torchvision.models.get_model(architecture)

In [18]:
full_model_dict = l
model_keys = [k for k in full_model_dict if 'attacker' not in k and 'normalizer' not in k]


In [19]:
model_dict = dict({k.split('module.model.')[-1]:full_model_dict[k] for k in model_keys})
model.load_state_dict(model_dict)

<All keys matched successfully>

In [20]:
normalizer_keys = [k for k in full_model_dict if 'attacker' not in k and 'normalizer' in k]
normalizer_dict = dict({k.split('_')[-1]:full_model_dict[k] for k in normalizer_keys})
normalizer = torchvision.transforms.Normalize(mean=normalizer_dict['mean'], std=normalizer_dict['std'])
    

In [23]:
encoder = torch.nn.Sequential(
        OrderedDict([
            ('normalizer',normalizer), 
            *list(model.named_children())[:-1], 
            ('flatten', torch.nn.Flatten())
        ])
    )

In [24]:
encoder.eval()

Sequential(
  (normalizer): Normalize(mean=tensor([[[0.4850]],
  
          [[0.4560]],
  
          [[0.4060]]]), std=tensor([[[0.2290]],
  
          [[0.2240]],
  
          [[0.2250]]]))
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e

In [27]:
encoder.eval().to(device)

Sequential(
  (normalizer): Normalize(mean=tensor([[[0.4850]],
  
          [[0.4560]],
  
          [[0.4060]]]), std=tensor([[[0.2290]],
  
          [[0.2240]],
  
          [[0.2250]]]))
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e

: 